# Matching

In [1]:
import numpy as np
import uproot
import vector
import awkward
import matplotlib.pyplot as plt
import mplhep as hep
import boost_histogram as bh
import itertools

directory = "/Users/archiebrooks/Documents/Uni/mphys project/"
#directory = "c:/Users/matis/OneDrive/Documents/Y4/Project/"

hep.style.use('ATLAS')

## Functions and Maps

In [2]:
def histogram(data, bins, data_label, axes, density=False, ratio=False, ratio_axes=None, set_range = None, weight_array=None, x_units='GeV'):
    if len(data[0]) != 1:
        if set_range is not None:
            global_min, global_max = set_range
        else:
            global_min = min([np.min(d) for d in data])
            global_max = max([np.max(d) for d in data])

        bin_edges = np.linspace(global_min, global_max, bins+1)
        counts = []
        errors = []
        bin_width = bin_edges[1]-bin_edges[0]
        for i in range(len(data)):
            if type(data[i])!= 'numpy.ndarray':
                data[i] = np.array(awkward.to_numpy(data[i]))
            hist = bh.Histogram(bh.axis.Regular(bins, global_min, global_max))
            hist.fill(data[i], weight=weight_array[i]) if weight_array is not None else hist.fill(data[i])
            norm_factor = np.sum(hist.counts() * np.diff(hist.axes[0].edges))
            if density: hep.histplot(hist.counts()/norm_factor, hist.axes[0].edges, ax=axes, yerr=np.sqrt(hist.variances())/norm_factor,label=data_label[i], histtype='step')
            else: hep.histplot(hist.counts(), hist.axes[0].edges, ax=axes, yerr=np.sqrt(hist.variances()),label=data_label, histtype='step')
            counts.append(hist.counts()/norm_factor) if density else counts.append(hist.counts())
            errors.append(np.sqrt(hist.variances())/norm_factor) if density else errors.append(np.sqrt(hist.variances()))       
    else:
        hist = bh.Histogram(bh.axis.Regular(bins, global_min, global_max))
        hist.fill(data[i])
        hep.histplot(hist.view(), hist.axes[0].edges, ax=axes, yerr=np.sqrt(hist.variances()), label=data_label, histtype='step')

    bin_width = hist.axes[0].edges[1]-hist.axes[0].edges[0]
    axes.set_ylabel(f'Events /{bin_width:.2g} {x_units}')
    axes.legend()

    if ratio:
        reference_counts = counts[0]
        for i in range(1, len(counts)):
            ratio_values = np.divide(counts[i], reference_counts, out=np.zeros_like(counts[i]), where=reference_counts != 0)
            ratio_errors = np.sqrt(np.divide(errors[i],counts[i], out=np.zeros_like(counts[i]), where=counts[i]!=0)**2 + (np.divide(errors[0],counts[0], out=np.zeros_like(counts[0]), where=counts[0]!=0)**2))
            hep.histplot(ratio_values, bin_edges, ax=ratio_axes, yerr=ratio_errors, label=f'{data_label[i]}/{data_label[0]}')
        ratio_axes.set_ylabel("Ratio")
        ratio_axes.set_xlabel(axes.get_xlabel())
        ratio_axes.axhline(1, color='black', linestyle='--')  # Reference line at ratio=1
        ratio_axes.legend()
        if ratio_axes.get_ylim()[1]>5:
            ratio_axes.set_ylim(0,5)

    hep.atlas.label(ax=axes)

def n_combinations(particles, n, dictionary, type):
    i = n
    while len(particles[awkward.num(particles)>=i])!=0:
        particles_i = particles[awkward.num(particles)==i]
        if len(particles_i)==0:
            i+=1
            continue
        indices = np.array(list(itertools.combinations(range(np.shape(particles_i)[1]), n)))
        dictionary[type+f'_{i}'] = particles_i[:, indices]
        i += 1
    return None

mass_lookup = {
    1: 2.16E-3,
    2: 4.7E-3,
    3: 1.273,
    4: 93.5E-3,
    5: 172.57,
    6: 4.183,
    11: 0.511E-3,
    12: 0,
    13: 105.66E-3,
    14: 0,
    15: 1.77693,
    16: 0,
    21: 0
}

## Data

In [ ]:
tree4 = uproot.open(directory + "4tops_withtruth_oct24.root")
events_4t_jets = tree4["Delphes;1"]["Jet"].arrays(['Jet.PT','Jet.Eta','Jet.Phi','Jet.Mass','Jet.BTag'])
events_4t_electrons = tree4["Delphes;1"]["Electron"].arrays(['Electron.PT','Electron.Eta','Electron.Phi'])
events_4t_muons = tree4["Delphes;1"]["Muon"].arrays(['Muon.PT','Muon.Eta','Muon.Phi'])
events_4t_neutrinos = tree4["Delphes;1"]['MissingET'].arrays(['MissingET.MET', 'MissingET.Eta', 'MissingET.Phi'])
events_4t_truth = tree4['Delphes;1']['Particle'].arrays(['Particle.PID','Particle.Status','Particle.Mass','Particle.PT','Particle.Eta','Particle.Phi'])
events_4t_truth = events_4t_truth[(events_4t_truth['Particle.Status']==23)]

events_4t_electrons['Electron.Mass'] = np.ones_like(events_4t_electrons['Electron.PT']) * 0.511E-3
events_4t_muons['Muon.Mass'] = np.ones_like(events_4t_muons['Muon.PT']) * 105.7E-3

jets_pt_4t = events_4t_jets['Jet.PT'][(events_4t_jets['Jet.Eta']<2.5) & (events_4t_jets['Jet.Eta']>-2.5)]
jets_eta_4t = events_4t_jets['Jet.Eta'][(events_4t_jets['Jet.Eta']<2.5) & (events_4t_jets['Jet.Eta']>-2.5)]
jets_phi_4t = events_4t_jets['Jet.Phi'][(events_4t_jets['Jet.Eta']<2.5) & (events_4t_jets['Jet.Eta']>-2.5)]
jets_mass_4t = events_4t_jets['Jet.Mass'][(events_4t_jets['Jet.Eta']<2.5) & (events_4t_jets['Jet.Eta']>-2.5)]

electrons_pt_4t = events_4t_electrons['Electron.PT'][(events_4t_electrons['Electron.PT'] > 15) & (events_4t_electrons['Electron.Eta'] < 2.47) & (events_4t_electrons['Electron.Eta'] > -2.47)]
electrons_eta_4t = events_4t_electrons['Electron.Eta'][(events_4t_electrons['Electron.PT'] > 15) & (events_4t_electrons['Electron.Eta'] < 2.47) & (events_4t_electrons['Electron.Eta'] > -2.47)]
electrons_phi_4t = events_4t_electrons['Electron.Phi'][(events_4t_electrons['Electron.PT'] > 15) & (events_4t_electrons['Electron.Eta'] < 2.47) & (events_4t_electrons['Electron.Eta'] > -2.47)]
electrons_mass_4t = events_4t_electrons['Electron.Mass'][(events_4t_electrons['Electron.PT'] > 15) & (events_4t_electrons['Electron.Eta'] < 2.47) & (events_4t_electrons['Electron.Eta'] > -2.47)]

muons_pt_4t = events_4t_muons['Muon.PT'][(events_4t_muons['Muon.PT']>15) & (events_4t_muons['Muon.Eta']<2.5) & (events_4t_muons['Muon.Eta']>-2.5)]
muons_eta_4t = events_4t_muons['Muon.Eta'][(events_4t_muons['Muon.PT']>15) & (events_4t_muons['Muon.Eta']<2.5) & (events_4t_muons['Muon.Eta']>-2.5)]
muons_phi_4t = events_4t_muons['Muon.Phi'][(events_4t_muons['Muon.PT']>15) & (events_4t_muons['Muon.Eta']<2.5) & (events_4t_muons['Muon.Eta']>-2.5)]
muons_mass_4t = events_4t_muons['Muon.Mass'][(events_4t_muons['Muon.PT']>15) & (events_4t_muons['Muon.Eta']<2.5) & (events_4t_muons['Muon.Eta']>-2.5)]

reco_pt_4t = awkward.concatenate((jets_pt_4t,electrons_pt_4t,muons_pt_4t), axis=1)
reco_eta_4t = awkward.concatenate((jets_eta_4t,electrons_eta_4t,muons_eta_4t), axis=1)
reco_phi_4t = awkward.concatenate((jets_phi_4t,electrons_phi_4t,muons_phi_4t), axis=1)
reco_mass_4t = awkward.concatenate((jets_mass_4t,electrons_mass_4t,muons_mass_4t), axis=1)

truth_pt_4t = events_4t_truth['Particle.PT']
truth_eta_4t = events_4t_truth['Particle.Eta']
truth_phi_4t = events_4t_truth['Particle.Phi']
truth_mass_4t = events_4t_truth['Particle.Mass']

reco_4t = vector.zip({'pt':reco_pt_4t,'eta':reco_eta_4t,'phi':reco_phi_4t,'mass':reco_mass_4t})
truth_4t = vector.zip({'pt':truth_pt_4t,'eta':truth_eta_4t,'phi':truth_phi_4t,'mass':truth_mass_4t})

In [ ]:
tree3j = uproot.open(directory + "4tops_withtruth_oct24.root")
events_4t_jets = tree4["Delphes;1"]["Jet"].arrays(['Jet.PT','Jet.Eta','Jet.Phi','Jet.Mass','Jet.BTag'])
events_4t_electrons = tree4["Delphes;1"]["Electron"].arrays(['Electron.PT','Electron.Eta','Electron.Phi'])
events_4t_muons = tree4["Delphes;1"]["Muon"].arrays(['Muon.PT','Muon.Eta','Muon.Phi'])
events_4t_neutrinos = tree4["Delphes;1"]['MissingET'].arrays(['MissingET.MET', 'MissingET.Eta', 'MissingET.Phi'])
events_4t_truth = tree4['Delphes;1']['Particle'].arrays(['Particle.PID','Particle.Status','Particle.Mass','Particle.PT','Particle.Eta','Particle.Phi'])
events_4t_truth = events_4t_truth[(events_4t_truth['Particle.Status']==23)]

events_4t_electrons['Electron.Mass'] = np.ones_like(events_4t_electrons['Electron.PT']) * 0.511E-3
events_4t_muons['Muon.Mass'] = np.ones_like(events_4t_muons['Muon.PT']) * 105.7E-3

jets_pt_4t = events_4t_jets['Jet.PT'][(events_4t_jets['Jet.Eta']<2.5) & (events_4t_jets['Jet.Eta']>-2.5)]
jets_eta_4t = events_4t_jets['Jet.Eta'][(events_4t_jets['Jet.Eta']<2.5) & (events_4t_jets['Jet.Eta']>-2.5)]
jets_phi_4t = events_4t_jets['Jet.Phi'][(events_4t_jets['Jet.Eta']<2.5) & (events_4t_jets['Jet.Eta']>-2.5)]
jets_mass_4t = events_4t_jets['Jet.Mass'][(events_4t_jets['Jet.Eta']<2.5) & (events_4t_jets['Jet.Eta']>-2.5)]

electrons_pt_4t = events_4t_electrons['Electron.PT'][(events_4t_electrons['Electron.PT'] > 15) & (events_4t_electrons['Electron.Eta'] < 2.47) & (events_4t_electrons['Electron.Eta'] > -2.47)]
electrons_eta_4t = events_4t_electrons['Electron.Eta'][(events_4t_electrons['Electron.PT'] > 15) & (events_4t_electrons['Electron.Eta'] < 2.47) & (events_4t_electrons['Electron.Eta'] > -2.47)]
electrons_phi_4t = events_4t_electrons['Electron.Phi'][(events_4t_electrons['Electron.PT'] > 15) & (events_4t_electrons['Electron.Eta'] < 2.47) & (events_4t_electrons['Electron.Eta'] > -2.47)]
electrons_mass_4t = events_4t_electrons['Electron.Mass'][(events_4t_electrons['Electron.PT'] > 15) & (events_4t_electrons['Electron.Eta'] < 2.47) & (events_4t_electrons['Electron.Eta'] > -2.47)]

muons_pt_4t = events_4t_muons['Muon.PT'][(events_4t_muons['Muon.PT']>15) & (events_4t_muons['Muon.Eta']<2.5) & (events_4t_muons['Muon.Eta']>-2.5)]
muons_eta_4t = events_4t_muons['Muon.Eta'][(events_4t_muons['Muon.PT']>15) & (events_4t_muons['Muon.Eta']<2.5) & (events_4t_muons['Muon.Eta']>-2.5)]
muons_phi_4t = events_4t_muons['Muon.Phi'][(events_4t_muons['Muon.PT']>15) & (events_4t_muons['Muon.Eta']<2.5) & (events_4t_muons['Muon.Eta']>-2.5)]
muons_mass_4t = events_4t_muons['Muon.Mass'][(events_4t_muons['Muon.PT']>15) & (events_4t_muons['Muon.Eta']<2.5) & (events_4t_muons['Muon.Eta']>-2.5)]

reco_pt_4t = awkward.concatenate((jets_pt_4t,electrons_pt_4t,muons_pt_4t), axis=1)
reco_eta_4t = awkward.concatenate((jets_eta_4t,electrons_eta_4t,muons_eta_4t), axis=1)
reco_phi_4t = awkward.concatenate((jets_phi_4t,electrons_phi_4t,muons_phi_4t), axis=1)
reco_mass_4t = awkward.concatenate((jets_mass_4t,electrons_mass_4t,muons_mass_4t), axis=1)

truth_pt_4t = events_4t_truth['Particle.PT']
truth_eta_4t = events_4t_truth['Particle.Eta']
truth_phi_4t = events_4t_truth['Particle.Phi']
truth_mass_4t = events_4t_truth['Particle.Mass']

reco_4t = vector.zip({'pt':reco_pt_4t,'eta':reco_eta_4t,'phi':reco_phi_4t,'mass':reco_mass_4t})
truth_4t = vector.zip({'pt':truth_pt_4t,'eta':truth_eta_4t,'phi':truth_phi_4t,'mass':truth_mass_4t})

In [251]:
reco_4t_test = reco_4t[(awkward.num(reco_4t)!=0)&(awkward.num(truth_4t)!=0)]
truth_4t_test = truth_4t[(awkward.num(reco_4t)!=0)&(awkward.num(truth_4t)!=0)]

truth_4t_12 = truth_4t[awkward.num(truth_4t)==12]
reco_4t_12 = reco_4t[awkward.num(truth_4t)==12]

truth_4t_13 = truth_4t[awkward.num(truth_4t)==13]
truth_4t_13 = truth_4t_13[:,1:]
reco_4t_13 = reco_4t[awkward.num(truth_4t)==13]

truth_4t_test = awkward.concatenate((truth_4t_12, truth_4t_13))
reco_4t_test = awkward.concatenate((reco_4t_12, reco_4t_13))
id_test = np.tile(np.arange(1,13), (61640, 1))

pairs_2d = awkward.cartesian({'reco':reco_4t_test,'truth':truth_4t_test})
id_pairs_2d = awkward.cartesian({'reco':reco_4t_test,'id':id_test})
pairs_3d = awkward.unflatten(pairs_2d, 12, axis=1)
id_pairs_3d = awkward.unflatten(id_pairs_2d, 12, axis=1)

deltars = pairs_3d['reco'].deltaR(pairs_3d['truth'])
min_deltars = np.min(deltars, axis=2)
id_pairs_3d = id_pairs_3d[(deltars == min_deltars)]
id_pairs_3d = id_pairs_3d[min_deltars<0.4]

top_1_particles = id_pairs_3d['reco'][(id_pairs_3d['id']==1)|(id_pairs_3d['id']==5)|(id_pairs_3d['id']==9)]
top_2_particles = id_pairs_3d['reco'][(id_pairs_3d['id']==2)|(id_pairs_3d['id']==6)|(id_pairs_3d['id']==10)]
antitop_1_particles = id_pairs_3d['reco'][(id_pairs_3d['id']==3)|(id_pairs_3d['id']==7)|(id_pairs_3d['id']==11)]
antitop_2_particles = id_pairs_3d['reco'][(id_pairs_3d['id']==4)|(id_pairs_3d['id']==8)|(id_pairs_3d['id']==12)]

top_1_particles = top_1_particles[awkward.num(top_1_particles, axis=2)!=0]
top_2_particles = top_2_particles[awkward.num(top_2_particles, axis=2)!=0]
antitop_1_particles = antitop_1_particles[awkward.num(antitop_1_particles, axis=2)!=0]
antitop_2_particles = antitop_2_particles[awkward.num(antitop_2_particles, axis=2)!=0]

<MomentumArray4D [[], [], [], [], ..., [], [], [], []] type='1650 * var * o...'>

In [262]:
truth_4t_test[0,8].deltaR(top_1_particles[0,0])

<Array [0.0646] type='1 * ?float32'>

In [220]:
min_deltars = np.min(deltars, axis=2)
print((min_deltars[0]))

[0.115, 0.128, 0.038, 0.0623, 0.0646, 0.035, 0.087, 0.0392, 0.127, 0.186]


In [115]:
reco_4t_test = reco_4t[(awkward.num(reco_4t)!=0)&(awkward.num(partonic_4t)!=0)]
partonic_4t_test = partonic_4t[(awkward.num(reco_4t)!=0)&(awkward.num(partonic_4t)!=0)]

print(len(reco_4t_test))

reco_counts = awkward.num(reco_4t_test, axis=1)
sorted_indices = awkward.argsort(reco_counts, ascending=False)
reco_4t_test = reco_4t_test[sorted_indices]
partonic_4t_test = partonic_4t_test[sorted_indices]

pairs_by_event = {}

while(len(reco_4t_test[awkward.num(reco_4t_test)!=0])!=0):
    pairs = awkward.cartesian({'reco':reco_4t_test,'parton':(partonic_4t_test)})
    deltars = pairs['reco'].deltaR(pairs['parton'])
    print(len(deltars))
    pairs = pairs[deltars == np.min(deltars, axis=1)]
    deltars = np.min(deltars, axis=1)

    pairs = pairs[deltars<0.4]

    partonic_4t_test = partonic_4t_test[awkward.num(reco_4t_test)>1]
    reco_4t_test = reco_4t_test[awkward.num(reco_4t_test)>1]
    reco_4t_test = reco_4t_test[:,1:]

300000
300000
299998
299980


KeyboardInterrupt: 

In [135]:
reco_4t_test = reco_4t[(awkward.num(reco_4t)!=0)&(awkward.num(partonic_4t)!=0)]
partonic_4t_test = partonic_4t[(awkward.num(reco_4t)!=0)&(awkward.num(partonic_4t)!=0)]



pairs = awkward.cartesian({'reco':reco_4t_test,'parton':(partonic_4t_test)})
deltars = pairs['reco'].deltaR(pairs['parton'])

deltars_3d = awkward.unflatten(deltars, awkward.num(partonic_4t_test), axis=2)

ValueError: structure imposed by 'counts' does not fit in the array or partition at axis=2

This error occurred while calling

    ak.unflatten(
        <Array [[2.98, 2.72, ..., 3.19, 2.84], ...] type='300000 * var * fl...'>
        <Array [11, 9, 10, 8, 8, ..., 11, 8, 11, 11, 12] type='300000 * int64'>
        axis = 2
    )

In [167]:
reco_4t_test = reco_4t[(awkward.num(reco_4t)!=0)&(awkward.num(partonic_4t)!=0)]
partonic_4t_test = partonic_4t[(awkward.num(reco_4t)!=0)&(awkward.num(partonic_4t)!=0)]

pairs = awkward.cartesian({'reco':reco_4t_test[:,0],'parton':(partonic_4t_test)})
deltars = pairs['reco'].deltaR(pairs['parton'])

pairs_2 = awkward.unflatten(pairs, awkward.num(reco_4t_test), axis=2)

ValueError: structure imposed by 'counts' does not fit in the array or partition at axis=2

This error occurred while calling

    ak.unflatten(
        <Array [[{reco: {...}, ...}, ...], ...] type='300000 * var * {reco:...'>
        <Array [11, 9, 9, 10, 8, 9, ..., 7, 8, 9, 8, 9] type='300000 * int64'>
        axis = 2
    )

In [173]:
truth_level_numbers = list(dict.fromkeys(awkward.num(partonic_4t_test)))

truth_4t_dict = {}
reco_4t_dict = {}

for truth_number in truth_level_numbers:
    truth_4t_dict[f'truth_particles_{truth_number}'] = partonic_4t_test[awkward.num(partonic_4t_test)==truth_number]
    reco_4t_dict[f'truth_particles_{truth_number}'] = reco_4t_test[awkward.num(partonic_4t_test)==truth_number]

for key in truth_4t_dict.keys():
    truth_particles = truth_4t_dict[key]
    reco_particles = reco_4t_dict[key]
    num = awkward.num(truth_particles)[0]
    pairs = awkward.cartesian({'truth':truth_particles,'reco':reco_particles})
    pairs = awkward.unflatten(pairs, num, axis=1)
    deltars = pairs['reco'].deltaR(pairs['truth'])
    print(np.shape(pairs['truth'][1]))
    pairs = pairs[deltars == np.min(deltars, axis=1)]
    deltars = np.min(deltars, axis=1)
    pairs = pairs[deltars<0.4]
    print('Next Key')

[10, 11]


ValueError: cannot broadcast nested list

This error occurred while calling

    numpy.equal.__call__(
        <Array [[[2.98, ..., 1.56], ...], ...] type='32831 * var * 11 * flo...'>
        <Array [[0.0397, 0.046, ..., 0.595], ...] type='32831 * 11 * ?float32'>
    )

In [160]:
print(truth_4t_dict.keys())

dict_keys(['truth_particles_{truth_number}'])


In [107]:
def n_combinations(recos, partons, dictionary, type):
    i = 0
    while len(recos[awkward.num(recos)>=i])!=0:
        recos_i = recos[awkward.num(recos)==i]
        partons_i = partons[awkward.num(recos)==i]
        if len(recos_i)==0:
            i+=1
            continue
        pairs = awkward.cartesian({'reco':recos_i,'parton':(partons_i)})
        deltars = pairs['reco'].deltaR(pairs['parton'])
        pairs = pairs[deltars == np.min(deltars, axis=1)]
        deltars = np.min(deltars, axis=1)
        pairs = pairs[deltars<0.4]
        i += 1
    return None

reco_counts = awkward.num(reco_4t, axis=1)
sorted_indices = awkward.argsort(reco_counts, ascending=False)
reco_4t = reco_4t[sorted_indices]
partonic_4t = partonic_4t[sorted_indices]

pairs_by_event = {}

while(len(reco_4t[awkward.num(reco_4t)!=0])!=0):
    pairs = awkward.cartesian({'reco':reco_4t[:,0],'parton':(partonic_4t)})
    print(reco_4t[:,0])
    deltars = pairs['reco'].deltaR(pairs['parton'])
    print(len(deltars))
    pairs = pairs[deltars == np.min(deltars, axis=1)]
    deltars = np.min(deltars, axis=1)

    pairs = pairs[deltars<0.4]

    partonic_4t = partonic_4t[awkward.num(reco_4t)>1]
    reco_4t = reco_4t[awkward.num(reco_4t)>1]
    reco_4t = reco_4t[:,1:]
    

      

In [42]:
x = np.array(([0,1,2,3],[4,5,1,7],[8,2,10,11], [2,3,4,1], [23,43,12,9]))
indices = np.argmin(x,axis=1)
print(x[(range(len(x)),indices)])

[0 1 2 1 9]


In [66]:
pairs = awkward.cartesian({'reco':reco_4t,'parton':(partonic_4t)}, axis=1)
deltars = pairs['reco'].deltaR(pairs['parton'])

[{reco: {rho: 190, phi: 2.01, ...}, parton: {...}}, ..., {reco: {...}, ...}]


In [67]:
pairs[0]

<Array [{reco: {...}, parton: {...}}, ...] type='121 * {reco: Momentum4D[rh...'>

In [98]:
valid_pairs = {}
num_events = len(deltars)
reco_lengths = awkward.num(reco_4t)
for i in range(len(deltars)):
    deltars_3d = awkward.unflatten(deltars[i], reco_lengths[i], axis=0)
    indices = awkward.argmin(deltars_3d, axis=1)
    indices = indices + (np.arange(len(indices)) * reco_lengths[i])
    indices = indices[np.min(deltars_3d, axis=1) < 0.4]
    valid_pairs[f'event_{i}'] = pairs[i][indices]
    if i%1000==0: print(f'{i} events comlpeted')

0 events comlpeted
1000 events comlpeted
2000 events comlpeted
3000 events comlpeted
4000 events comlpeted
5000 events comlpeted
6000 events comlpeted
7000 events comlpeted
8000 events comlpeted
9000 events comlpeted
10000 events comlpeted
11000 events comlpeted
12000 events comlpeted
13000 events comlpeted
14000 events comlpeted
15000 events comlpeted


KeyboardInterrupt: 

In [100]:
import awkward as ak
import numpy as np

# Precompute reco_lengths for all events and ensure alignment
reco_lengths = ak.num(reco_4t)
if len(deltars) != ak.sum(reco_lengths):
    deltars = deltars[:ak.sum(reco_lengths)]  # Adjust if necessary

# Now perform unflattening and other calculations as previously outlined
deltars_3d = ak.unflatten(deltars, reco_lengths, axis=1)
min_values = ak.min(deltars_3d, axis=2)
indices = ak.argmin(deltars_3d, axis=2)
event_indices = ak.local_index(min_values) * reco_lengths
filtered_indices = indices + event_indices
mask = min_values < 0.4
filtered_indices = filtered_indices[mask]
valid_pairs = pairs[mask][filtered_indices]

print("Vectorized event processing completed.")


ValueError: structure imposed by 'counts' does not fit in the array or partition at axis=1

This error occurred while calling

    ak.unflatten(
        <Array [[2.98, 2.72, ..., 3.19, 2.84], ...] type='300000 * var * fl...'>
        <Array [11, 9, 9, 10, 8, 9, ..., 7, 8, 9, 8, 9] type='300000 * int64'>
        axis = 1
    )

In [95]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import awkward as ak
import numpy as np

def process_event(i):
    """Process an individual event to determine valid pairs."""
    deltars_3d = ak.unflatten(deltars[i], ak.num(reco_4t)[i], axis=0)
    indices = ak.argmin(deltars_3d, axis=1)
    indices = indices + (np.arange(len(indices)) * ak.num(reco_4t)[i])
    indices = indices[ak.min(deltars_3d, axis=1) < 0.4]
    valid_pairs = {f'event_{i}': pairs[i][indices]}
    return i, valid_pairs

def parallel_process_events():
    """Parallel processing of events using ProcessPoolExecutor."""
    valid_pairs = {}
    with ProcessPoolExecutor() as executor:
        # Submit tasks to the executor
        futures = {executor.submit(process_event, i): i for i in range(len(deltars))}
        
        # Collect results as they complete
        for future in as_completed(futures):
            i, result = future.result()
            valid_pairs.update(result)
            
            # Optional status update
            if i % 1000 == 0:
                print(f'{i} events completed')
    
    return valid_pairs

# Run the parallel event processing
valid_pairs = parallel_process_events()


Process SpawnProcess-4:
Process SpawnProcess-1:
Process SpawnProcess-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/archiebrooks/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/archiebrooks/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/archiebrooks/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/archiebrooks/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/archiebrooks/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/archiebrooks/opt/anaconda3/lib/python3.9/concurrent/futures/process.py", line 237, in _process_worker
    call_item = call_queue.get(block=T

BrokenProcessPool: A child process terminated abruptly, the process pool is not usable anymore

In [93]:
valid_pairs['event_1']['reco'].deltaR(valid_pairs['event_1']['parton'])

<Array [0.0314, 0.0848, 0.0275, 0.152, 0.0486] type='5 * ?float32'>

In [203]:
deltars_by_event = {}
pairs_by_event = {}
for i in range(1,23):
    recos = reco_4t[awkward.num(reco_4t)==i]
    while(len(reco_4t[awkward.num(reco_4t)!=0])!=0):
        pairs = awkward.cartesian({'reco':recos[:,0],'parton':(partonic_4t)})
        deltars = pairs['reco'].deltaR(pairs['parton'])
        pairs = pairs[deltars == np.min(deltars, axis=1)]
        deltars = np.min(deltars, axis=1)
        pairs = pairs[deltars<0.4]

        partonic_4t = partonic_4t[awkward.num(reco_4t)>1]
        reco_4t = reco_4t[awkward.num(reco_4t)>1]
        reco_4t = reco_4t[:,1:]


ValueError: cannot broadcast RegularArray of size 2 with RegularArray of size 300000

This error occurred while calling

    ak.cartesian(
        {'reco': <MomentumArray4D [{rho: 50.3, phi: -0.404, ...}, ...] type='...
    )

In [ ]:
deltars_by_event = {}
pairs_by_event = {}
for i in range(1,23):
    recos = reco_4t[awkward.num(reco_4t)==i]
    pairs = awkward.cartesian({'reco':recos[:,0],'parton':(partonic_4t)}, axis=0)
    print(i)


1
2
3
4


In [170]:
reco_counts = awkward.num(reco_4t, axis=1)
sorted_indices = awkward.argsort(reco_counts, ascending=False)
reco_4t = reco_4t[sorted_indices]
partonic_4t = partonic_4t[sorted_indices]

all_pairs = awkward.cartesian({"reco": reco_4t, "parton": partonic_4t}, axis=1)
all_deltars = all_pairs['reco'].deltaR(all_pairs['parton'])

awkward.unflatten(all_deltars[0], awkward.num(reco_4t)[0])

In [196]:
all_deltars_3d = awkward.Array(
    [awkward.unflatten(event, awkward.num(reco_4t[i])) for i, event in enumerate(all_deltars)]
)

AxisError: axis=1 exceeds the depth of this array (1)

This error occurred while calling

    ak.num(
        <MomentumArray4D [{rho: 460, phi: 1.13, ...}, ...] type='22 * Momen...'>
    )

In [158]:
pairs = awkward.cartesian({'reco':reco_4t[:,0],'parton':(partonic_4t)})
deltars = pairs['reco'].deltaR(pairs['parton'])
pairs = pairs[deltars == np.min(deltars, axis=1)]
deltars = np.min(deltars, axis=1)

pairs = pairs[deltars<0.4]
deltars = deltars[deltars<0.4]

In [159]:
pairs['reco'].deltaR(pairs['parton'])

<Array [[0.0397], [0.16], ..., [0.0553]] type='263667 * option[var * float32]'>

In [157]:
deltars

<Array [0.0397, 0.591, 0.16, ..., 0.0837, 0.0553] type='300000 * ?float32'>

In [134]:
reco_4t[0,0].deltaR(partonic_4t[0])
reco_4t[0,1].deltaR(partonic_4t[0])

<Array [1.05, 1.62, 2.43, 0.046, ..., 2.75, 3.06, 2.5] type='11 * float32'>

In [126]:
np.min(awkward.unflatten(deltars[0],awkward.num(reco_4t)[0],axis=0), axis=1)


<Array [0.78, 0.0261, 0.0948, ..., 0.0397, 0.262, 0.0367] type='11 * ?float32'>

In [73]:
pairs[0][deltars[0]<0.4]['reco']

<MomentumArray4D [{rho: 43.6, phi: -1.86, ...}, ...] type='12 * Momentum4D[...'>

In [84]:
len(muons_pt_4t[2])

0

In [56]:
pairs = awkward.cartesian({"parton": partonic_4t[0], "reco": reco_4t[0]}, axis=0)
print(len(pairs))
deltars = pairs['parton'].deltaR(pairs['reco'])
print(len(deltars))

121
121


In [6]:
tree3j = uproot.open(directory + "3tops_tttj_skimmed_combined.root")
tree3W = uproot.open(directory + "3tops_tttW_skimmed_combined.root")
tree4 = uproot.open(directory + "4tops_inclusive_17july_combined.root")

events_4t_jets = tree4["Delphes;1"]["Jet"].arrays(['Jet.PT','Jet.Eta','Jet.Phi','Jet.T','Jet.Mass','Jet.Flavor','Jet.BTag','Jet.Charge'])
events_4t_electrons = tree4["Delphes;1"]["Electron"].arrays(['Electron.PT','Electron.Eta','Electron.Phi','Electron.T','Electron.Charge','Electron.Particle'])
events_4t_electrons['Electron.Mass'] = np.ones_like(events_4t_electrons['Electron.PT']) * 0.511E-3
events_4t_muons = tree4["Delphes;1"]["Muon"].arrays(['Muon.PT','Muon.Eta','Muon.Phi','Muon.T','Muon.Charge','Muon.Particle'])
events_4t_muons['Muon.Mass'] = np.ones_like(events_4t_muons['Muon.PT']) * 105.7E-3

jets_pt_4t = events_4t_jets['Jet.PT'][(events_4t_jets['Jet.Eta']<2.5) & (events_4t_jets['Jet.Eta']>-2.5)]
jets_eta_4t = events_4t_jets['Jet.Eta'][(events_4t_jets['Jet.Eta']<2.5) & (events_4t_jets['Jet.Eta']>-2.5)]
jets_phi_4t = events_4t_jets['Jet.Phi'][(events_4t_jets['Jet.Eta']<2.5) & (events_4t_jets['Jet.Eta']>-2.5)]
jets_mass_4t = events_4t_jets['Jet.Mass'][(events_4t_jets['Jet.Eta']<2.5) & (events_4t_jets['Jet.Eta']>-2.5)]

electrons_pt_4t = events_4t_electrons['Electron.PT'][(events_4t_electrons['Electron.PT'] > 15) & (events_4t_electrons['Electron.Eta'] < 2.47) & (events_4t_electrons['Electron.Eta'] > -2.47)]
electrons_eta_4t = events_4t_electrons['Electron.Eta'][(events_4t_electrons['Electron.PT'] > 15) & (events_4t_electrons['Electron.Eta'] < 2.47) & (events_4t_electrons['Electron.Eta'] > -2.47)]
electrons_phi_4t = events_4t_electrons['Electron.Phi'][(events_4t_electrons['Electron.PT'] > 15) & (events_4t_electrons['Electron.Eta'] < 2.47) & (events_4t_electrons['Electron.Eta'] > -2.47)]
electrons_mass_4t = events_4t_electrons['Electron.Mass'][(events_4t_electrons['Electron.PT'] > 15) & (events_4t_electrons['Electron.Eta'] < 2.47) & (events_4t_electrons['Electron.Eta'] > -2.47)]
electrons_charge_4t = events_4t_electrons['Electron.Charge'][(events_4t_electrons['Electron.PT'] > 15) & (events_4t_electrons['Electron.Eta'] < 2.47) & (events_4t_electrons['Electron.Eta'] > -2.47)]

muons_pt_4t = events_4t_muons['Muon.PT'][(events_4t_muons['Muon.PT']>15) & (events_4t_muons['Muon.Eta']<2.5) & (events_4t_muons['Muon.Eta']>-2.5)]
muons_eta_4t = events_4t_muons['Muon.Eta'][(events_4t_muons['Muon.PT']>15) & (events_4t_muons['Muon.Eta']<2.5) & (events_4t_muons['Muon.Eta']>-2.5)]
muons_phi_4t = events_4t_muons['Muon.Phi'][(events_4t_muons['Muon.PT']>15) & (events_4t_muons['Muon.Eta']<2.5) & (events_4t_muons['Muon.Eta']>-2.5)]
muons_mass_4t = events_4t_muons['Muon.Mass'][(events_4t_muons['Muon.PT']>15) & (events_4t_muons['Muon.Eta']<2.5) & (events_4t_muons['Muon.Eta']>-2.5)]
muons_charge_4t = events_4t_muons['Muon.Charge'][(events_4t_muons['Muon.PT']>15) & (events_4t_muons['Muon.Eta']<2.5) & (events_4t_muons['Muon.Eta']>-2.5)]

electrons_pt_4t = electrons_pt_4t[awkward.num(muons_pt_4t)==2]
electrons_eta_4t = electrons_eta_4t[awkward.num(muons_eta_4t)==2]
electrons_phi_4t = electrons_phi_4t[awkward.num(muons_phi_4t)==2]
electrons_mass_4t = electrons_mass_4t[awkward.num(muons_mass_4t)==2]
electrons_charges_4t = electrons_charge_4t[awkward.num(muons_charge_4t)==2]

jets_pt_4t = jets_pt_4t[awkward.num(muons_pt_4t)==2]
jets_eta_4t = jets_eta_4t[awkward.num(muons_eta_4t)==2]
jets_phi_4t = jets_phi_4t[awkward.num(muons_phi_4t)==2]
jets_mass_4t = jets_mass_4t[awkward.num(muons_mass_4t)==2]

muons_pt_4t = muons_pt_4t[awkward.num(muons_pt_4t)==2]
muons_eta_4t = muons_eta_4t[awkward.num(muons_eta_4t)==2]
muons_phi_4t = muons_phi_4t[awkward.num(muons_phi_4t)==2]
muons_mass_4t = muons_mass_4t[awkward.num(muons_mass_4t)==2]
muons_charges_4t = muons_charge_4t[awkward.num(muons_charge_4t)==2]

leptons_pt_4t = awkward.concatenate((electrons_pt_4t, muons_pt_4t), axis=1)
leptons_eta_4t = awkward.concatenate((electrons_eta_4t, muons_eta_4t), axis=1)
leptons_phi_4t = awkward.concatenate((electrons_phi_4t, muons_phi_4t), axis=1)
leptons_mass_4t = awkward.concatenate((electrons_mass_4t, muons_mass_4t), axis=1)
leptons_charge_4t = awkward.concatenate((electrons_charges_4t, muons_charges_4t), axis=1)

jets_4t = vector.zip({'pt':jets_pt_4t,'eta':jets_eta_4t,'phi':jets_phi_4t,'mass':jets_mass_4t})
leptons_4t = vector.zip({'pt':leptons_pt_4t,'eta':leptons_eta_4t,'phi':leptons_phi_4t,'mass':leptons_mass_4t})

events_3tj_jets = tree3j["Delphes;1"]["Jet"].arrays(['Jet.PT', 'Jet.Eta', 'Jet.Phi', 'Jet.Mass', 'Jet.BTag'])
events_3tj_electrons = tree3j["Delphes;1"]["Electron"].arrays(['Electron.PT','Electron.Eta','Electron.Phi','Electron.Charge','Electron.Particle'])
events_3tj_electrons['Electron.Mass'] = np.ones_like(events_3tj_electrons['Electron.PT'])*0.511E-3
events_3tj_muons = tree3j["Delphes;1"]["Muon"].arrays(['Muon.PT', 'Muon.Eta', 'Muon.Phi', 'Muon.Charge', 'Muon.Particle'])
events_3tj_muons['Muon.Mass'] = np.ones_like(events_3tj_muons['Muon.PT'])*105.7E-3

events_3tW_jets = tree3W["Delphes;1"]["Jet"].arrays(['Jet.PT', 'Jet.Eta', 'Jet.Phi', 'Jet.Mass', 'Jet.BTag'])
events_3tW_electrons = tree3W["Delphes;1"]["Electron"].arrays(['Electron.PT','Electron.Eta','Electron.Phi','Electron.Charge','Electron.Particle'])
events_3tW_electrons['Electron.Mass'] = np.ones_like(events_3tW_electrons['Electron.PT'])*0.511E-3
events_3tW_muons = tree3W["Delphes;1"]["Muon"].arrays(['Muon.PT', 'Muon.Eta', 'Muon.Phi', 'Muon.Charge', 'Muon.Particle'])
events_3tW_muons['Muon.Mass'] = np.ones_like(events_3tW_muons['Muon.PT'])*105.7E-3

jets_pt_3tj = events_3tj_jets['Jet.PT'][(events_3tj_jets['Jet.Eta']<2.5) & (events_3tj_jets['Jet.Eta']>-2.5)]
jets_eta_3tj = events_3tj_jets['Jet.Eta'][(events_3tj_jets['Jet.Eta']<2.5) & (events_3tj_jets['Jet.Eta']>-2.5)]
jets_phi_3tj = events_3tj_jets['Jet.Phi'][(events_3tj_jets['Jet.Eta']<2.5) & (events_3tj_jets['Jet.Eta']>-2.5)]
jets_mass_3tj = events_3tj_jets['Jet.Mass'][(events_3tj_jets['Jet.Eta']<2.5) & (events_3tj_jets['Jet.Eta']>-2.5)]

electrons_pt_3tj = events_3tj_electrons['Electron.PT'][(events_3tj_electrons['Electron.PT']>15)&(events_3tj_electrons['Electron.Eta']<2.47)&(events_3tj_electrons['Electron.Eta']>-2.47)]
electrons_eta_3tj = events_3tj_electrons['Electron.Eta'][(events_3tj_electrons['Electron.PT']>15)&(events_3tj_electrons['Electron.Eta']<2.47)&(events_3tj_electrons['Electron.Eta']>-2.47)]
electrons_phi_3tj = events_3tj_electrons['Electron.Phi'][(events_3tj_electrons['Electron.PT']>15)&(events_3tj_electrons['Electron.Eta']<2.47)&(events_3tj_electrons['Electron.Eta']>-2.47)]
electrons_mass_3tj = events_3tj_electrons['Electron.Mass'][(events_3tj_electrons['Electron.PT']>15)&(events_3tj_electrons['Electron.Eta']<2.47)&(events_3tj_electrons['Electron.Eta']>-2.47)]
electrons_charge_3tj = events_3tj_electrons['Electron.Charge'][(events_3tj_electrons['Electron.PT']>15)&(events_3tj_electrons['Electron.Eta']<2.47)&(events_3tj_electrons['Electron.Eta']>-2.47)]

muons_pt_3tj = events_3tj_muons['Muon.PT'][(events_3tj_muons['Muon.PT']>15)&(events_3tj_muons['Muon.Eta']<2.5)&(events_3tj_muons['Muon.Eta']>-2.5)]
muons_eta_3tj = events_3tj_muons['Muon.Eta'][(events_3tj_muons['Muon.PT']>15)&(events_3tj_muons['Muon.Eta']<2.5)&(events_3tj_muons['Muon.Eta']>-2.5)]
muons_phi_3tj = events_3tj_muons['Muon.Phi'][(events_3tj_muons['Muon.PT']>15)&(events_3tj_muons['Muon.Eta']<2.5)&(events_3tj_muons['Muon.Eta']>-2.5)]
muons_mass_3tj = events_3tj_muons['Muon.Mass'][(events_3tj_muons['Muon.PT']>15)&(events_3tj_muons['Muon.Eta']<2.5)&(events_3tj_muons['Muon.Eta']>-2.5)]
muons_charge_3tj = events_3tj_muons['Muon.Charge'][(events_3tj_muons['Muon.PT']>15)&(events_3tj_muons['Muon.Eta']<2.5)&(events_3tj_muons['Muon.Eta']>-2.5)]

electrons_pt_3tj = electrons_pt_3tj[awkward.num(muons_pt_3tj)==2]
electrons_eta_3tj = electrons_eta_3tj[awkward.num(muons_eta_3tj)==2]
electrons_phi_3tj = electrons_phi_3tj[awkward.num(muons_phi_3tj)==2]
electrons_mass_3tj = electrons_mass_3tj[awkward.num(muons_mass_3tj)==2]
electrons_charge_3tj = electrons_charge_3tj[awkward.num(muons_charge_3tj)==2]

jets_pt_3tj = jets_pt_3tj[awkward.num(muons_pt_3tj)==2]
jets_eta_3tj = jets_eta_3tj[awkward.num(muons_eta_3tj)==2]
jets_phi_3tj = jets_phi_3tj[awkward.num(muons_phi_3tj)==2]
jets_mass_3tj = jets_mass_3tj[awkward.num(muons_mass_3tj)==2]

muons_pt_3tj = muons_pt_3tj[awkward.num(muons_pt_3tj)==2]
muons_eta_3tj = muons_eta_3tj[awkward.num(muons_eta_3tj)==2]
muons_phi_3tj = muons_phi_3tj[awkward.num(muons_phi_3tj)==2]
muons_mass_3tj = muons_mass_3tj[awkward.num(muons_mass_3tj)==2]
muons_charge_3tj = muons_charge_3tj[awkward.num(muons_charge_3tj)==2]

jets_pt_3tW = events_3tW_jets['Jet.PT'][(events_3tW_jets['Jet.Eta']<2.5) & (events_3tW_jets['Jet.Eta']>-2.5)]
jets_eta_3tW = events_3tW_jets['Jet.Eta'][(events_3tW_jets['Jet.Eta']<2.5) & (events_3tW_jets['Jet.Eta']>-2.5)]
jets_phi_3tW = events_3tW_jets['Jet.Phi'][(events_3tW_jets['Jet.Eta']<2.5) & (events_3tW_jets['Jet.Eta']>-2.5)]
jets_mass_3tW = events_3tW_jets['Jet.Mass'][(events_3tW_jets['Jet.Eta']<2.5) & (events_3tW_jets['Jet.Eta']>-2.5)]

electrons_pt_3tW = events_3tW_electrons['Electron.PT'][(events_3tW_electrons['Electron.PT']>15)&(events_3tW_electrons['Electron.Eta']<2.47)&(events_3tW_electrons['Electron.Eta']>-2.47)]
electrons_eta_3tW = events_3tW_electrons['Electron.Eta'][(events_3tW_electrons['Electron.PT']>15)&(events_3tW_electrons['Electron.Eta']<2.47)&(events_3tW_electrons['Electron.Eta']>-2.47)]
electrons_phi_3tW = events_3tW_electrons['Electron.Phi'][(events_3tW_electrons['Electron.PT']>15)&(events_3tW_electrons['Electron.Eta']<2.47)&(events_3tW_electrons['Electron.Eta']>-2.47)]
electrons_mass_3tW = events_3tW_electrons['Electron.Mass'][(events_3tW_electrons['Electron.PT']>15)&(events_3tW_electrons['Electron.Eta']<2.47)&(events_3tW_electrons['Electron.Eta']>-2.47)]
electrons_charge_3tW = events_3tW_electrons['Electron.Charge'][(events_3tW_electrons['Electron.PT']>15)&(events_3tW_electrons['Electron.Eta']<2.47)&(events_3tW_electrons['Electron.Eta']>-2.47)]

muons_pt_3tW = events_3tW_muons['Muon.PT'][(events_3tW_muons['Muon.PT']>15)&(events_3tW_muons['Muon.Eta']<2.5)&(events_3tW_muons['Muon.Eta']>-2.5)]
muons_eta_3tW = events_3tW_muons['Muon.Eta'][(events_3tW_muons['Muon.PT']>15)&(events_3tW_muons['Muon.Eta']<2.5)&(events_3tW_muons['Muon.Eta']>-2.5)]
muons_phi_3tW = events_3tW_muons['Muon.Phi'][(events_3tW_muons['Muon.PT']>15)&(events_3tW_muons['Muon.Eta']<2.5)&(events_3tW_muons['Muon.Eta']>-2.5)]
muons_mass_3tW = events_3tW_muons['Muon.Mass'][(events_3tW_muons['Muon.PT']>15)&(events_3tW_muons['Muon.Eta']<2.5)&(events_3tW_muons['Muon.Eta']>-2.5)]
muons_charge_3tW = events_3tW_muons['Muon.Charge'][(events_3tW_muons['Muon.PT']>15)&(events_3tW_muons['Muon.Eta']<2.5)&(events_3tW_muons['Muon.Eta']>-2.5)]

electrons_pt_3tW = electrons_pt_3tW[awkward.num(muons_pt_3tW)==2]
electrons_eta_3tW = electrons_eta_3tW[awkward.num(muons_eta_3tW)==2]
electrons_phi_3tW = electrons_phi_3tW[awkward.num(muons_phi_3tW)==2]
electrons_mass_3tW = electrons_mass_3tW[awkward.num(muons_mass_3tW)==2]
electrons_charge_3tW = electrons_charge_3tW[awkward.num(muons_charge_3tW)==2]

jets_pt_3tW = jets_pt_3tW[awkward.num(muons_pt_3tW)==2]
jets_eta_3tW = jets_eta_3tW[awkward.num(muons_eta_3tW)==2]
jets_phi_3tW = jets_phi_3tW[awkward.num(muons_phi_3tW)==2]
jets_mass_3tW = jets_mass_3tW[awkward.num(muons_mass_3tW)==2]

muons_pt_3tW = muons_pt_3tW[awkward.num(muons_pt_3tW)==2]
muons_eta_3tW = muons_eta_3tW[awkward.num(muons_eta_3tW)==2]
muons_phi_3tW = muons_phi_3tW[awkward.num(muons_phi_3tW)==2]
muons_mass_3tW = muons_mass_3tW[awkward.num(muons_mass_3tW)==2]
muons_charge_3tW = muons_charge_3tW[awkward.num(muons_charge_3tW)==2]

leptons_pt_3tW = awkward.concatenate([electrons_pt_3tW,muons_pt_3tW],axis=1)
leptons_eta_3tW = awkward.concatenate([electrons_eta_3tW,muons_eta_3tW],axis=1)
leptons_phi_3tW = awkward.concatenate([electrons_phi_3tW,muons_phi_3tW],axis=1)
leptons_mass_3tW = awkward.concatenate([electrons_mass_3tW,muons_mass_3tW],axis=1)
leptons_charge_3tW = awkward.concatenate([electrons_charge_3tW,muons_charge_3tW],axis=1)

leptons_pt_3tj = awkward.concatenate([electrons_pt_3tj,muons_pt_3tj],axis=1)
leptons_eta_3tj = awkward.concatenate([electrons_eta_3tj,muons_eta_3tj],axis=1)
leptons_phi_3tj = awkward.concatenate([electrons_phi_3tj,muons_phi_3tj],axis=1)
leptons_mass_3tj = awkward.concatenate([electrons_mass_3tj,muons_mass_3tj],axis=1)
leptons_charge_3tj = awkward.concatenate([electrons_charge_3tj,muons_charge_3tj],axis=1)

leptons_pt_3t = awkward.concatenate([leptons_pt_3tW,leptons_pt_3tj])
leptons_eta_3t = awkward.concatenate([leptons_eta_3tW,leptons_eta_3tj])
leptons_phi_3t = awkward.concatenate([leptons_phi_3tW,leptons_phi_3tj])
leptons_mass_3t = awkward.concatenate([leptons_mass_3tW,leptons_mass_3tj])
leptons_charge_3t = awkward.concatenate([leptons_charge_3tW,leptons_charge_3tj])

jets_pt_3t = awkward.concatenate([jets_pt_3tW,jets_pt_3tj])
jets_eta_3t = awkward.concatenate([jets_eta_3tW,jets_eta_3tj])
jets_phi_3t = awkward.concatenate([jets_phi_3tW,jets_phi_3tj])
jets_mass_3t = awkward.concatenate([jets_mass_3tW,jets_mass_3tj])

leptons_3t = vector.zip({'pt':leptons_pt_3t,'eta':leptons_eta_3t,'phi':leptons_phi_3t,'mass':leptons_mass_3t})
jets_3t = vector.zip({'pt':jets_pt_3t,'eta':jets_eta_3t,'phi':jets_phi_3t,'mass':jets_mass_3t})

filtered_jets_3t = jets_3t[(awkward.num(leptons_3t)>=2) & (awkward.num(jets_3t)>=3)]
filtered_leptons_3t = leptons_3t[(awkward.num(jets_3t)>=3) & (awkward.num(leptons_3t)>=2)]
filtered_leptons_charge_3t = leptons_charge_3t[(awkward.num(jets_3t)>=3) & (awkward.num(leptons_3t)>=2)]

jets_3t = filtered_jets_3t
leptons_3t = filtered_leptons_3t
leptons_charge_3t = filtered_leptons_charge_3t

filtered_jets_4t = jets_4t[(awkward.num(leptons_4t)>=2) & (awkward.num(jets_4t)>=3)]
filtered_leptons_4t = leptons_4t[(awkward.num(jets_4t)>=3) & (awkward.num(leptons_4t)>=2)]
filtered_leptons_charge_4t = leptons_charge_4t[(awkward.num(jets_4t)>=3) & (awkward.num(leptons_4t)>=2)]

jets_4t = filtered_jets_4t
leptons_4t = filtered_leptons_4t
leptons_charge_4t = filtered_leptons_charge_4t

all_products_4t = awkward.concatenate((jets_4t,leptons_4t), axis=1)
all_products_3t = awkward.concatenate((jets_3t,leptons_3t), axis=1)

In [9]:
jet_duos_3t = {}
jet_duos_4t = {}
jet_trios_3t = {}
jet_trios_4t = {}
jet_quads_3t = {}
jet_quads_4t = {}
jet_5_3t = {}
jet_5_4t = {}
jet_6_3t = {}
jet_6_4t = {}
n_combinations(jets_3t, 2, jet_duos_3t, 'jets')
n_combinations(jets_4t, 2, jet_duos_4t, 'jets')
n_combinations(jets_3t, 3, jet_trios_3t, 'jets')
n_combinations(jets_4t, 3, jet_trios_4t, 'jets')
n_combinations(jets_3t, 4, jet_quads_3t, 'jets')
n_combinations(jets_4t, 4, jet_quads_4t, 'jets')
n_combinations(jets_3t, 5, jet_5_3t, 'jets')
n_combinations(jets_4t, 5, jet_5_4t, 'jets')
n_combinations(jets_3t, 6, jet_6_3t, 'jets')
n_combinations(jets_4t, 6, jet_6_4t, 'jets')

lepton_duos_3t = {}
lepton_duos_4t = {}
lepton_trios_3t = {}
lepton_trios_4t = {}
lepton_quads_3t = {}
lepton_quads_4t = {}
n_combinations(leptons_3t, 2, lepton_duos_3t, 'leptons')
n_combinations(leptons_4t, 2, lepton_duos_4t, 'leptons')
n_combinations(leptons_3t, 3, lepton_trios_3t, 'leptons')
n_combinations(leptons_4t, 3, lepton_trios_4t, 'leptons')
n_combinations(leptons_3t, 4, lepton_quads_3t, 'leptons')
n_combinations(leptons_4t, 4, lepton_quads_4t, 'leptons')

In [10]:
jet_duos_3t

{'jets_3': <MomentumArray4D [[[{rho: 78.7, ...}, ...], ...], ...] type='2112 * 3 * 2 *...'>,
 'jets_4': <MomentumArray4D [[[{rho: 108, ...}, ...], ...], ...] type='4028 * 6 * 2 * ...'>,
 'jets_5': <MomentumArray4D [[[{rho: 346, ...}, ...], ...], ...] type='4843 * 10 * 2 *...'>,
 'jets_6': <MomentumArray4D [[[{rho: 166, ...}, ...], ...], ...] type='4018 * 15 * 2 *...'>,
 'jets_7': <MomentumArray4D [[[{rho: 157, ...}, ...], ...], ...] type='2267 * 21 * 2 *...'>,
 'jets_8': <MomentumArray4D [[[{rho: 229, ...}, ...], ...], ...] type='1067 * 28 * 2 *...'>,
 'jets_9': <MomentumArray4D [[[{rho: 201, ...}, ...], ...], ...] type='323 * 36 * 2 * ...'>,
 'jets_10': <MomentumArray4D [[[{rho: 141, ...}, ...], ...], ...] type='99 * 45 * 2 * M...'>,
 'jets_11': <MomentumArray4D [[[{rho: 177, ...}, ...], ...], ...] type='29 * 55 * 2 * M...'>,
 'jets_12': <MomentumArray4D [[[{rho: 320, ...}, ...], ...], ...] type='5 * 66 * 2 * Mo...'>,
 'jets_13': <MomentumArray4D [[[{rho: 517, ...}, {...}], ..., [...]

In [13]:
print(list(itertools.product(('A','B','C','D'),('x','y'))))

[('A', 'x'), ('A', 'y'), ('B', 'x'), ('B', 'y'), ('C', 'x'), ('C', 'y'), ('D', 'x'), ('D', 'y')]
